In [1]:
import mediapipe as mp
import cv2
import numpy as np
import math
import uuid
import os
import pyautogui

In [2]:
print(cv2.__version__)

4.8.1


In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Simple Code to show camera frames

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Video Output', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# A fully functional code showing the overlay of fingers on camera output frames

In [6]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # # Detections
        # print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=10),
                                        mp_drawing.DrawingSpec(color=(198, 44, 250), thickness=2, circle_radius=5),
                                         )
            
        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### Print the Landmarks of the hands

In [13]:
print(hands) #connected with the previous cell

<module 'mediapipe.python.solutions.hands' from 'c:\\Users\\kunal\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\mediapipe\\python\\solutions\\hands.py'>


# Code which changes the color of the finger tips

In [7]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hand Module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils
# Custom colors for fingertips
finger_tip_colors = [(0, 0, 255),  # Red for thumb
                     (0, 255, 0),  # Green for index finger
                     (255, 0, 0),  # Blue for middle finger
                     (255, 255, 0),  # Yellow for ring finger
                     (0, 255, 255)  # Cyan for pinky finger
                    ]

# Function to get fingertip coordinates for a hand
def get_fingertip_coordinates(landmarks):
    fingertip_coordinates = []
    for idx, landmark in enumerate(landmarks.landmark):
        if idx in [4, 8, 12, 16, 20]:  # Indexes for the tips of the five fingers
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            fingertip_coordinates.append((cx, cy))
    return fingertip_coordinates

# Capture Video Stream (or Load Image)
cap = cv2.VideoCapture(0)  # 0 for the default webcam

while True:
    ret, frame = cap.read()  # Read a frame from the webcam

    # Convert the BGR frame to RGB (MediaPipe uses RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get fingertip coordinates for the hand
            fingertip_coords = get_fingertip_coordinates(hand_landmarks)

            # Draw circles for fingertips with custom colors
            for i, coord in enumerate(fingertip_coords):
                color = finger_tip_colors[i]
                cv2.circle(frame, coord, radius=10, color=color, thickness=-1)  # Draw a filled circle

            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the frame with landmarks
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):  # Press Q to exit
        break

cap.release()
cv2.destroyAllWindows()


# Code with detection of gestures

In [4]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hand Module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Custom colors for fingertips
finger_tip_colors = [(0, 0, 255),  # Red for thumb
                     (0, 255, 0),  # Green for index finger
                     (255, 0, 0),  # Blue for middle finger
                     (255, 255, 0),  # Yellow for ring finger
                     (0, 255, 255)  # Cyan for pinky finger
                    ]

# Function to get fingertip coordinates for a hand
def get_fingertip_coordinates(landmarks):
    fingertip_coordinates = []
    for idx, landmark in enumerate(landmarks.landmark):
        if idx in [4, 8, 12, 16, 20]:  # Indexes for the tips of the five fingers
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            fingertip_coordinates.append((cx, cy))
    return fingertip_coordinates

# Function to detect and print gestures
def detect_gestures(landmarks):
    # Calculate distances between fingertip landmarks
    thumb_tip = landmarks[0]
    index_tip = landmarks[1]
    middle_tip = landmarks[2]
    ring_tip = landmarks[3]
    pinky_tip = landmarks[4]

    # Calculate Euclidean distances between thumb and other fingertips
    distance_to_index = ((thumb_tip[0] - index_tip[0])**2 + (thumb_tip[1] - index_tip[1])**2)**0.5
    distance_to_middle = ((thumb_tip[0] - middle_tip[0])**2 + (thumb_tip[1] - middle_tip[1])**2)**0.5
    distance_to_ring = ((thumb_tip[0] - ring_tip[0])**2 + (thumb_tip[1] - ring_tip[1])**2)**0.5
    distance_to_pinky = ((thumb_tip[0] - pinky_tip[0])**2 + (thumb_tip[1] - pinky_tip[1])**2)**0.5

    # Define a threshold for the fist gesture
    fist_threshold = 60.0  # Adjust this threshold as needed

    # Detect and print gestures
    if (distance_to_index < fist_threshold and
        distance_to_middle < fist_threshold and
        distance_to_ring < fist_threshold and
        distance_to_pinky < fist_threshold):
        return "Fist"
    else:
        return "Open Hand"

# Capture Video Stream (or Load Image)
cap = cv2.VideoCapture(0)  # 0 for the default webcam

while True:
    ret, frame = cap.read()  # Read a frame from the webcam

    # Convert the BGR frame to RGB (MediaPipe uses RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)
    

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get fingertip coordinates for the hand
            fingertip_coords = get_fingertip_coordinates(hand_landmarks)

            # Draw circles for fingertips with custom colors
            for i, coord in enumerate(fingertip_coords):
                color = finger_tip_colors[i]
                cv2.circle(frame, coord, radius=10, color=color, thickness=-1)  # Draw a filled circle

            # Detect and print gestures
            gesture = detect_gestures(fingertip_coords)
            cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the frame with landmarks and gestures after "RESIZE"
    resize = cv2.resize(frame, (1280, 960))
    cv2.imshow("Hand Tracking", resize)

    if cv2.waitKey(10) & 0xFF == ord('q'): # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


# Moving Curser with Open Hand and stopping with fist

In [6]:
import cv2
import mediapipe as mp
import pyautogui
import threading
pyautogui.FAILSAFE = False
# Initialize MediaPipe Hand Module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

finger_tip_colors = [(0, 0, 255),  # Red for thumb
                     (0, 255, 0),  # Green for index finger
                     (255, 0, 0),  # Blue for middle finger
                     (255, 255, 0),  # Yellow for ring finger
                     (0, 255, 255)  # Cyan for pinky finger
                    ]
# Constants for screen dimensions (adjust as needed)
SCREEN_WIDTH = 1920  # Screen width in pixels
SCREEN_HEIGHT = 1080  # Screen height in pixels

# Function to get fingertip coordinates for a hand
def get_fingertip_coordinates(landmarks):
    fingertip_coordinates = []
    for idx, landmark in enumerate(landmarks.landmark):
        if idx in [4, 8, 12, 16, 20]:  # Indexes for the tips of the five fingers
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            fingertip_coordinates.append((cx, cy))
    return fingertip_coordinates

# Function to detect and print gestures
def detect_gestures(landmarks):
    # Calculate distances between fingertip landmarks
    thumb_tip = landmarks[0]
    index_tip = landmarks[1]
    middle_tip = landmarks[2]
    ring_tip = landmarks[3]
    pinky_tip = landmarks[4]

    # Calculate Euclidean distances between thumb and other fingertips
    distance_to_index = ((thumb_tip[0] - index_tip[0])**2 + (thumb_tip[1] - index_tip[1])**2)**0.5
    distance_to_middle = ((thumb_tip[0] - middle_tip[0])**2 + (thumb_tip[1] - middle_tip[1])**2)**0.5
    distance_to_ring = ((thumb_tip[0] - ring_tip[0])**2 + (thumb_tip[1] - ring_tip[1])**2)**0.5
    distance_to_pinky = ((thumb_tip[0] - pinky_tip[0])**2 + (thumb_tip[1] - pinky_tip[1])**2)**0.5

    # Define a threshold for the open hand gesture
    open_hand_threshold_cm = 80.0  # Adjust this threshold as needed
    fist_threshold_cm = 100.0  # Adjust this threshold as needed

    # Detect open hand gesture
    if (
        distance_to_index > open_hand_threshold_cm
        and distance_to_middle > open_hand_threshold_cm
        and distance_to_ring > open_hand_threshold_cm
        and distance_to_pinky > open_hand_threshold_cm
    ):
        return "Open Hand"
    # Detect fist gesture
    elif (
        distance_to_index < fist_threshold_cm
        and distance_to_middle < fist_threshold_cm
        and distance_to_ring < fist_threshold_cm
        and distance_to_pinky < fist_threshold_cm
    ):
        return "Fist"
    else:
        return "Neutral"

# Function to control the mouse cursor relative to the screen
def control_mouse(fingertip_coords, last_cursor_position):
    if len(fingertip_coords) > 0:
        x, y = pyautogui.position()  # Get the current mouse position
        speed = 30  # Adjust the speed as needed
        movement_distance_cm = 4.0  # Adjust the desired movement distance in centimeters
        movement_distance_px = (movement_distance_cm / SCREEN_WIDTH) * SCREEN_WIDTH
        dx = fingertip_coords[0][0] - last_cursor_position[0]
        dy = fingertip_coords[0][1] - last_cursor_position[1]
        pyautogui.moveRel(dx * movement_distance_px, dy * movement_distance_px, duration=0.1 / speed)
        return (fingertip_coords[0][0], fingertip_coords[0][1])
    return last_cursor_position

# Capture Video Stream (or Load Image)
cap = cv2.VideoCapture(0)  # 0 for the default webcam

# Initialize the cursor position
cursor_position = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)

while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    # image = cv2.flip(image, 1)
    # Convert the BGR frame to RGB (MediaPipe uses RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)
    close=0
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get fingertip coordinates for the hand
            fingertip_coords = get_fingertip_coordinates(hand_landmarks)

            # Draw circles for fingertips with custom colors
            for i, coord in enumerate(fingertip_coords):
                color = finger_tip_colors[i]
                cv2.circle(frame, coord, radius=10, color=color, thickness=-1)  # Draw a filled circle

            # Detect and print gestures
            gesture = detect_gestures(fingertip_coords)
            cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Control the mouse cursor based on the open hand gesture (in a separate thread)
            if gesture == "Open Hand":
                cursor_position = control_mouse(fingertip_coords, cursor_position)
            if gesture == "Fist":
                # pyautogui.click()

                close=1
                break

            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        if close==1:
            break
    if close==1:
        break


    # Display the frame with landmarks and gestures
    # image = cv2.flip(image, 1)
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


# Added Scrolling functionality using Index fingers


In [11]:
import cv2
import mediapipe as mp
import pyautogui
import threading
pyautogui.FAILSAFE = False
# Initialize MediaPipe Hand Module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

finger_tip_colors = [(0, 0, 255),  # Red for thumb
                     (0, 255, 0),  # Green for index finger
                     (255, 0, 0),  # Blue for middle finger
                     (255, 255, 0),  # Yellow for ring finger
                     (0, 255, 255)  # Cyan for pinky finger
                    ]
# Constants for screen dimensions (adjust as needed)
SCREEN_WIDTH = 1920  # Screen width in pixels
SCREEN_HEIGHT = 1080  # Screen height in pixels
SCROLL_SPEED = 20  # Adjust the scroll speed as needed
SCROLL_THRESHOLD = 100  # Adjust the scroll threshold as needed
# Function to get fingertip coordinates for a hand
def get_fingertip_coordinates(landmarks):
    fingertip_coordinates = []
    for idx, landmark in enumerate(landmarks.landmark):
        if idx in [4, 8, 12, 16, 20]:  # Indexes for the tips of the five fingers
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            fingertip_coordinates.append((cx, cy))
    return fingertip_coordinates

# Function to detect and print gestures
def detect_gestures(landmarks):
    # Calculate distances between fingertip landmarks
    thumb_tip = landmarks[0]
    index_tip = landmarks[1]
    middle_tip = landmarks[2]
    ring_tip = landmarks[3]
    pinky_tip = landmarks[4]

    # Calculate Euclidean distances between thumb and other fingertips
    distance_to_index = ((thumb_tip[0] - index_tip[0])**2 + (thumb_tip[1] - index_tip[1])**2)**0.5
    distance_to_middle = ((thumb_tip[0] - middle_tip[0])**2 + (thumb_tip[1] - middle_tip[1])**2)**0.5
    distance_to_ring = ((thumb_tip[0] - ring_tip[0])**2 + (thumb_tip[1] - ring_tip[1])**2)**0.5
    distance_to_pinky = ((thumb_tip[0] - pinky_tip[0])**2 + (thumb_tip[1] - pinky_tip[1])**2)**0.5

    # Define a threshold for the open hand gesture
    open_hand_threshold_cm = 50.0  # Adjust this threshold as needed
    fist_threshold_cm = 50.0  # Adjust this threshold as needed

    # Detect open hand gesture
    if (
        distance_to_index > open_hand_threshold_cm
        and distance_to_middle > open_hand_threshold_cm
        and distance_to_ring > open_hand_threshold_cm
        and distance_to_pinky > open_hand_threshold_cm
    ):
        return "Open Hand"
    # Detect fist gesture
    elif (
        distance_to_index < fist_threshold_cm
        and distance_to_middle < fist_threshold_cm
        and distance_to_ring < fist_threshold_cm
        and distance_to_pinky < fist_threshold_cm
    ):
        return "Fist"
    # Get the y-coordinate of the index fingertip
    index_fingertip_y = landmarks[0][1] if len(landmarks) > 0 else None

    # Initialize previous fingertip y-coordinate
    if "prev_y" not in detect_gestures.__dict__:
        detect_gestures.prev_y = index_fingertip_y

    # Calculate the difference in y-coordinate
    y_difference = index_fingertip_y - detect_gestures.prev_y
    detect_gestures.prev_y = index_fingertip_y

    # Detect scrolling gestures
    if y_difference > SCROLL_THRESHOLD:
        return "Scroll Down"
    elif y_difference < -SCROLL_THRESHOLD:
        return "Scroll Up"

    return "Neutral"
def perform_scroll(gesture):
    if gesture == "Scroll Up":
        pyautogui.scroll(SCROLL_SPEED)
    elif gesture == "Scroll Down":
        pyautogui.scroll(-SCROLL_SPEED)

# Function to control the mouse cursor relative to the screen
def control_mouse(fingertip_coords, last_cursor_position):
    if len(fingertip_coords) > 0:
        x, y = pyautogui.position()  # Get the current mouse position
        speed = 30  # Adjust the speed as needed
        movement_distance_cm = 4.0  # Adjust the desired movement distance in centimeters
        movement_distance_px = (movement_distance_cm / SCREEN_WIDTH) * SCREEN_WIDTH
        dx = fingertip_coords[0][0] - last_cursor_position[0]
        dy = fingertip_coords[0][1] - last_cursor_position[1]
        pyautogui.moveRel(dx * movement_distance_px, dy * movement_distance_px, duration=0.1 / speed)
        return (fingertip_coords[0][0], fingertip_coords[0][1])
    return last_cursor_position

# Capture Video Stream (or Load Image)
cap = cv2.VideoCapture(0)  # 0 for the default webcam

# Initialize the cursor position
cursor_position = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)

while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    # image = cv2.flip(image, 1)
    # Convert the BGR frame to RGB (MediaPipe uses RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)
    close=0
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get fingertip coordinates for the hand
            fingertip_coords = get_fingertip_coordinates(hand_landmarks)

            # Draw circles for fingertips with custom colors
            for i, coord in enumerate(fingertip_coords):
                color = finger_tip_colors[i]
                cv2.circle(frame, coord, radius=10, color=color, thickness=-1)  # Draw a filled circle

            # Detect and print gestures
            gesture = detect_gestures(fingertip_coords)
            cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Control the mouse cursor based on the open hand gesture (in a separate thread)
            # Perform scrolling based on gestures
            if gesture=="Scroll Down" or gesture=="Scroll Up":
                perform_scroll(gesture)
            if gesture == "Open Hand":
                cursor_position = control_mouse(fingertip_coords, cursor_position)
            # if gesture == "Fist":
            #     pyautogui.click()

            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the frame with landmarks and gestures
    # image = cv2.flip(image, 1)
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


# Scrolling using index finger

In [8]:
import cv2
import mediapipe as mp
import pyautogui

# Disable PyAutoGUI fail-safe
pyautogui.FAILSAFE = False

# Initialize MediaPipe Hand Module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Constants for screen dimensions (adjust as needed)
SCREEN_WIDTH = 1920  # Screen width in pixels
SCREEN_HEIGHT = 1080  # Screen height in pixels

# Function to get fingertip coordinates for a hand
def get_fingertip_coordinates(landmarks):
    fingertip_coordinates = []
    for idx, landmark in enumerate(landmarks.landmark):
        if idx == 8:  # Index for the tip of the index finger
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            fingertip_coordinates.append((cx, cy))
    return fingertip_coordinates

# Function to control the cursor based on finger movement
def control_cursor(fingertip_coords, last_cursor_position):
    if len(fingertip_coords) > 0:
        x, y = pyautogui.position()  # Get the current cursor position
        speed = 20  # Adjust the speed as needed
        movement_distance_cm = 5.0  # Adjust the desired movement distance in centimeters
        movement_distance_px = (movement_distance_cm / SCREEN_WIDTH) * SCREEN_WIDTH
        dx = fingertip_coords[0][0] - last_cursor_position[0]
        dy = fingertip_coords[0][1] - last_cursor_position[1]
        pyautogui.moveRel(dx * movement_distance_px, dy * movement_distance_px, duration=0.1 / speed)
        return (fingertip_coords[0][0], fingertip_coords[0][1])
    return last_cursor_position

# Capture Video Stream (or Load Image)
cap = cv2.VideoCapture(0)  # 0 for the default webcam

# Initialize the cursor position
cursor_position = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)

while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    frame=cv2.flip(frame,1)

    # Convert the BGR frame to RGB (MediaPipe uses RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get fingertip coordinates for the hand
            fingertip_coords = get_fingertip_coordinates(hand_landmarks)

            # Draw a circle for the index fingertip
            if fingertip_coords:
                cv2.circle(frame, fingertip_coords[0], radius=10, color=(0, 255, 0), thickness=-1)  # Green circle

                # Control the cursor based on finger movement
                cursor_position = control_cursor(fingertip_coords, cursor_position)

            # Draw hand landmarks
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the frame with landmarks and cursor movement
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


# Moving cursor with index finger and clicking by folding middle finger

In [5]:
import cv2
import mediapipe as mp
import pyautogui
import math

# Disable PyAutoGUI fail-safe
pyautogui.FAILSAFE = False

# Initialize MediaPipe Hand Module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Constants for screen dimensions (adjust as needed)
SCREEN_WIDTH = 1920  # Screen width in pixels
SCREEN_HEIGHT = 1080  # Screen height in pixels
prev_fingertip_coords = None
# Constants for cursor movement smoothing
# SMOOTHING_FACTOR = 0.2  Adjust the smoothing factor (0.0 to 1.0) as needed

# Variables to track cursor position and velocity
cursor_position = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
cursor_velocity = [0, 0]

# Custom colors for fingertips
finger_tip_colors = [(0, 0, 255),  # Red for thumb
                     (0, 255, 0),  # Green for index finger
                     (255, 0, 0),  # Blue for middle finger
                     (255, 255, 0),  # Yellow for ring finger
                     (0, 255, 255)  # Cyan for pinky finger
                    ]

# Function to get fingertip coordinates for a hand
def get_fingertip_coordinates(landmarks):
    fingertip_coordinates = []
    for idx, landmark in enumerate(landmarks.landmark):
        if idx ==8:  # Indexes for the tips of the five fingers
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            fingertip_coordinates.append((cx, cy))
    return fingertip_coordinates

 
# Function to control the cursor based on finger movement
movement_distance_cm = 9.0  # Adjust this value as needed
# ... (previous code)

# Variables for low-pass filtering
filter_alpha = 1.0  # Smoothing factor (0.0 to 1.0)
# ... (previous code)

# Function to get distance between two points
def calculate_distance(point1, point2):
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** 0.5

# ... (previous code)

# Function to check if the middle finger is folded
def calculate_angle(l1, l2, l3):
    x1, y1 = l1.x, l1.y
    x2, y2 = l2.x, l2.y
    x3, y3 = l3.x, l3.y

    # Calculate the vectors between the landmarks
    vec1 = (x1 - x2, y1 - y2)
    vec2 = (x3 - x2, y3 - y2)

    # Calculate the dot product and the magnitudes of the vectors
    dot_product = vec1[0] * vec2[0] + vec1[1] * vec2[1]
    magnitude1 = (vec1[0] ** 2 + vec1[1] ** 2) ** 0.5
    magnitude2 = (vec2[0] ** 2 + vec2[1] ** 2) ** 0.5

    # Calculate the angle in degrees
    angle_rad = math.acos(dot_product / (magnitude1 * magnitude2))
    angle_deg = math.degrees(angle_rad)

    return angle_deg
def is_middle_finger_folded(hand_landmarks):
    # Get the landmarks of the middle finger
    middle_finger_landmarks = [
        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP],
        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP],
        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP],
        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ]

    # Calculate the angle between the finger joints
    angle1 = calculate_angle(middle_finger_landmarks[0], middle_finger_landmarks[1], middle_finger_landmarks[2])

    # Define a threshold angle (adjust as needed)
    threshold_angle = 90.0  # Adjust this value to control the sensitivity

    # Check if the middle finger is folded (angle is less than the threshold)
    return angle1 < threshold_angle

    # Check if the middle finger is folded
    return all(angle < 0 for angle in angles)

# Function to control cursor movement and click
def control_cursor_and_click(fingertip_coords, hand_landmarks):
    global cursor_position, cursor_velocity, prev_fingertip_coords

    if len(fingertip_coords) >= 1:
        # Check if the middle finger is folded
        if is_middle_finger_folded(hand_landmarks):
            pyautogui.click()  # Execute a click action
        else:
            # Move the cursor with the index finger
            if prev_fingertip_coords is not None:
                dx = fingertip_coords[0][0] - prev_fingertip_coords[0][0]
                dy = fingertip_coords[0][1] - prev_fingertip_coords[0][1]

                movement_x = dx * movement_distance_cm * SCREEN_WIDTH / 1920
                movement_y = dy * movement_distance_cm * SCREEN_HEIGHT / 1080

                cursor_velocity = (
                    filter_alpha * movement_x + (1 - filter_alpha) * cursor_velocity[0],
                    filter_alpha * movement_y + (1 - filter_alpha) * cursor_velocity[1],
                )

                cursor_position = (
                    cursor_position[0] + int(cursor_velocity[0]),
                    cursor_position[1] + int(cursor_velocity[1]),
                )

                cursor_position = (
                    min(max(cursor_position[0], 0), SCREEN_WIDTH),
                    min(max(cursor_position[1], 0), SCREEN_HEIGHT),
                )

                pyautogui.moveTo(cursor_position[0], cursor_position[1])

        prev_fingertip_coords = fingertip_coords
    else:
        prev_fingertip_coords = None

# Capture Video Stream (or Load Image)
cap = cv2.VideoCapture(0)  # 0 for the default webcam

while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    frame=cv2.flip(frame,1)
    # Convert the BGR frame to RGB (MediaPipe uses RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get fingertip coordinates for the hand
            fingertip_coords = get_fingertip_coordinates(hand_landmarks)

            # Draw circles for fingertips with custom colors
            for i, coord in enumerate(fingertip_coords):
                color = finger_tip_colors[i]
                cv2.circle(frame, coord, radius=10, color=color, thickness=-1)  # Draw a filled circle
            
                # Control the cursor based on finger movement
                control_cursor_and_click(fingertip_coords, hand_landmarks)
            if is_middle_finger_folded(hand_landmarks):
                    cv2.putText(frame, "Middle Finger Folded", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw hand landmarks
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the frame with landmarks and smoothed cursor movement
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from pynput.keyboard import Key, Controller as KeyboardController
from pynput.mouse import  Controller as MouseController

mouse = MouseController()
keyboard=KeyboardController()

k=int(input("enter number"))
if k==1:
    
